# DynaPIN: Dynamic Analysis of Protein INterfaces - Interactive Notebook

**DynaPIN** allows for the analysis of protein-protein interactions, structural stability, and energetic contributions from MD simulations.

This notebook demonstrates how to use the DynaPIN Python API directly, bypassing the command line interface.

**Prerequisites:**
1. Ensure the `dynapin` environment is activated.
2. If you want to calculate energies, ensure you have the path to your **FoldX** executable.

### Step 1: Import Libraries
First, we import the necessary modules from the DynaPIN package and setup the display environment.

In [ ]:
import os
import warnings

# Import DynaPIN core modules
from DynaPIN.Tables import dynapin
from DynaPIN.Plots import Plotter

# Import IPython display tools to view results inline
from IPython.display import Image, display
warnings.filterwarnings('ignore')

print("DynaPIN modules loaded successfully.")

### Step 2: Configuration

Here we define the input parameters required for the analysis. 

**Input Arguments:**

* `trajectory_file`: Path to the input simulation file (`.pdb` or `.dcd`).
* `topology_file`: Path to the topology file (e.g., `.psf`). Required **only** if the trajectory is a DCD file; otherwise, set to `None`.
* `job_name`: Name of the output directory where results will be stored.
* `stride`: Step size for reading frames (e.g., `1` reads every frame, `10` reads every 10th frame). Increase this for faster testing.
* `split_models`: Set to `True` for multi-frame PDBs to ensure they are treated as a trajectory.
* `chains`: A list of specific chains to analyze (e.g., `['A', 'B']`). Set to `None` to let DynaPIN auto-detect interactions.
* `foldx_exe`: Absolute path to the **FoldX** executable. Set to `None` if you do not wish to run energetic analysis.

In [ ]:
# Input Files
trajectory_file = "my_simulation.pdb" 
topology_file = None

# Output Settings
job_name = "analysis_output_01"
output_dir = os.path.abspath(job_name)

# Analysis Parameters
stride = 1 
split_models = True
chains = None 
foldx_exe = "/path/to/foldx" 

### Step 3: Visualize Trajectory
Before running the analysis, we use **NGLView** to visualize the trajectory. 

In [ ]:
import nglview as nv
import MDAnalysis as mda

if topology_file:
    u_viz = mda.Universe(topology_file, trajectory_file)
else:
    u_viz = mda.Universe(trajectory_file)

print(f"Loaded: {len(u_viz.atoms)} atoms, {len(u_viz.trajectory)} frames.")


view = nv.show_mdanalysis(u_viz.atoms)
view.clear_representations()
view.add_representation("cartoon", selection="protein", color="chainid")
view.add_representation("ball+stick", selection="not protein and not water", color="element")
view.center()
view.camera = "orthographic"

view

### Step 4: Initialize Analysis Engine
We initialize the main `dynapin` class. This setup process creates the output directory, processes the input trajectory (handling chains, water/ion removal), and prepares the system for specific analysis modules.

In [ ]:
print(f"Initializing DynaPIN analysis for: {trajectory_file}")

mol = dynapin(
    trajectory_file=trajectory_file,
    stride=stride,
    split_models=split_models,
    chains=chains,
    output_dir=output_dir,
    topology_file=topology_file,
    remove_water=True,
    remove_ions=True
)

print(f"Setup complete. Output directory created at: {mol.output_dir}")

### Step 5: Quality Control (QC)
This module calculates structural stability metrics including:
* **RMSD**: Root Mean Square Deviation (Global stability)
* **Rg**: Radius of Gyration (Compactness)
* **RMSF**: Root Mean Square Fluctuation (Flexibility)
* **Interface Metrics**: iRMSD, lRMSD, DockQ, Fnat, Fnonnat

In [ ]:
# Configuration: Use first frame (0) as reference structure
rmsd_config = {
    'ref_struc': None, 
    'ref_frame': 0
}

print("Running Quality Control (RMSD, RG, RMSF, DockQ)...")
mol.run_quality_control(rmsd_data=rmsd_config)
print("Quality Control analysis complete.")

### Step 6: Residue-Based Analysis
This module performs residue-level calculations:
* **SASA**: Solvent Accessible Surface Area
* **Biophysical Classification**: Core, Rim, Support, Interior, Surface
* **DSSP**: Secondary structure assignment
* **FoldX**: Energetic contributions (Van der Waals, Electrostatics, etc.) if `foldx_exe` is provided.
  
*Note: This step can be computationally intensive for large trajectories.*

In [ ]:
print("Running Residue-Based Analysis...")

if foldx_exe is None:
    print("Note: FoldX path not provided. Skipping energy calculations.")

mol.run_res_based(foldx_path=foldx_exe)
print("Residue-Based analysis complete.")

#### Visualize Dynamic Interface Residues

Here we visualize the residues identified as significant interface contributors (`Interface Score > 50`). 

* **Cartoon (Transparent):** The overall protein structure.
* **Sticks (Solid):** The dynamic interface residues highlighted.

In [ ]:
import pandas as pd
import glob
import re
import os
import MDAnalysis as mda
import nglview as nv
from IPython.display import display

pdb_files = glob.glob(os.path.join(output_dir, "*_chained_standardized.pdb"))
if not pdb_files:
    print("Standardized PDB not found. Using input trajectory for visualization.")
    viz_structure = trajectory_file
else:
    viz_structure = pdb_files[0]
    print(f"Visualizing Dynamic Interfaces on: {os.path.basename(viz_structure)}")

stats_path = os.path.join(output_dir, "tables", "res_interface_stats.csv")
if not os.path.exists(stats_path):
    print("Interface stats file not found. Ensure Residue-Based Analysis ran successfully.")
else:
    df_stats = pd.read_csv(stats_path)
    interface_res = df_stats[df_stats['Interface_score'] > 50.0]
    selection_parts = []
    for _, row in interface_res.iterrows():
        res_str = row['Residue'] # e.g. ALA10
        chain = row['Chain']
        res_num = re.findall(r'\d+', str(res_str))
        if res_num:
            selection_parts.append(f"{res_num[0]}:{chain}")
            
    selection_string = " or ".join(selection_parts)
    
    u_int = mda.Universe(viz_structure)
    print(f"Loaded {len(u_int.atoms)} atoms. Creating view...")
    
    view_int = nv.show_mdanalysis(u_int.atoms)
    
    view_int.clear_representations()
    
    view_int.add_representation("cartoon", selection="protein", color="chainid", opacity=0.2)
    if selection_string:
        print(f"Highlighting {len(selection_parts)} residues.")
        view_int.add_representation("ball+stick", selection=selection_string, color="chainid")
    else:
        print("No residues found with Interface Score > 50.")
        
    view_int.center()
    
    display(view_int)

### Step 7: Interaction-Based Analysis
This module maps specific atomic interactions over the simulation:
* Hydrogen Bonds
* Hydrophobic Interactions
* Ionic Bonds (Salt Bridges)

*Note: This step can be computationally intensive for large trajectories.*

In [ ]:
print("Running Interaction-Based Analysis...")
print("This may take a while depending on trajectory size...")

mol.run_inter_based(get_all_hph=False)
print("Interaction-Based analysis complete.")

### Step 8: Generate Plots
We use the `Plotter` class to visualize the calculated data. The plots are saved as high-quality PNG files in the `figures/` subdirectory.

In [ ]:
print(f"Generating plots in: {os.path.join(output_dir, 'figures')}")

plotter = Plotter(output_dir=output_dir)

# QC Plots
plotter.plot_rmsd()
plotter.plot_rg()
plotter.plot_rmsf()
plotter.plot_irmsd()
plotter.plot_dockq()

# Residue Plots
plotter.plot_biophys()
plotter.plot_SASA()
plotter.plot_DSSP(threshold=50.0)

# Interaction Plots
plotter.plot_pairwise_freq()

# Energy Plots (if FoldX was run)
if foldx_exe:
    plotter.plot_int_energy(threshold=50.0)

print("All plots generated successfully.")

### Step 9: Interactive Result Viewer
Finally, we display the generated plots directly within the notebook.

In [ ]:
figure_path = os.path.join(output_dir, "figures")

def show_plot(filename):
    path = os.path.join(figure_path, filename)
    if os.path.exists(path):
        print(f"--- {filename} ---")
        display(Image(filename=path, width=600))
    else:
        print(f"Plot not found: {filename}")

# Display selection of key plots
show_plot("qc_rmsd_backbone.png")
show_plot("qc_dockq_score.png")
show_plot("res_biophys_composition.png")
show_plot("int_pairwise_hbond-frequency.png")